In [3]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.1.2'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:5 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:6 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:13 https://developer.download.nvidia.com/

In [4]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-06-20 23:13:30--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.67MB/s    in 0.6s    

2021-06-20 23:13:31 (1.67 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [6]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Watches_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.head(5)

[Row(marketplace='US', customer_id=3653882, review_id='R3O9SGZBVQBV76', product_id='B00FALQ1ZC', product_parent=937001370, product_title='Invicta Women\'s 15150 "Angel" 18k Yellow Gold Ion-Plated Stainless Steel and Brown Leather Watch', product_category='Watches', star_rating=5, helpful_votes=0, total_votes=0, vine='N', verified_purchase='Y', review_headline='Five Stars', review_body='Absolutely love this watch! Get compliments almost every time I wear it. Dainty.', review_date='2015-08-31'),
 Row(marketplace='US', customer_id=14661224, review_id='RKH8BNC3L5DLF', product_id='B00D3RGO20', product_parent=484010722, product_title="Kenneth Cole New York Women's KC4944 Automatic Silver Automatic Mesh Bracelet Analog Watch", product_category='Watches', star_rating=5, helpful_votes=0, total_votes=0, vine='N', verified_purchase='Y', review_headline='I love thiswatch it keeps time wonderfully', review_body='I love this watch it keeps time wonderfully.', review_date='2015-08-31'),
 Row(marketpl

### Create DataFrames to match tables

In [7]:
# Read in the Review dataset as a DataFrame
watches_df = df
watches_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    3653882|R3O9SGZBVQBV76|B00FALQ1ZC|     937001370|Invicta Women's 1...|         Watches|          5|            0|          0|   N|                Y|          Five Stars|Absolutely love t...| 2015-08-31|
|         US|   14661224| RKH8BNC3L5DLF|B00D3RGO20|     484010722|Kenneth Cole New ...|         Watches|          5|    

In [71]:
# Create the vine_table. DataFrame
vine_df = watches_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3O9SGZBVQBV76|          5|            0|          0|   N|                Y|
| RKH8BNC3L5DLF|          5|            0|          0|   N|                Y|
|R2HLE8WKZSU3NL|          2|            1|          1|   N|                Y|
|R31U3UH5AZ42LL|          5|            0|          0|   N|                Y|
|R2SV659OUJ945Y|          4|            0|          0|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [73]:
#filter out all the rows where total_votes >= 20
vine_df_20plus = vine_df.filter(vine_df.total_votes >= 20)
vine_df_20plus.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R14W2VCHHK5V7W|          5|           19|         20|   N|                Y|
|R1S3T57O3OYT5S|          5|           19|         20|   N|                Y|
|R1BTWIBLYYVOV7|          5|           30|         30|   N|                Y|
| R6F9VY91ADPLA|          1|            8|         30|   N|                N|
|R3PXNV89DFIXKV|          5|           35|         37|   N|                Y|
|R2ZF9NYVT3J7D6|          5|           19|         22|   N|                Y|
|R20NYA5V0UF9NE|          5|           27|         28|   N|                Y|
|R2X8FZRUOS8R8C|          4|           25|         26|   N|                Y|
|R2D8IMBVX3XCLF|          1|           14|         20|   N|                Y|
|R25UD9TA63L3Q8|          5|           25|         27|   N|     

In [81]:
#filter out all the helpful_votes/total_votes >= 0.5
#create a new column percentage of helpful votes
vine_helpful_comments_df = vine_df_20plus.withColumn('fraction_helpful_votes',(vine_df_20plus.helpful_votes)/(vine_df_20plus.total_votes))
vine_helpful_comments_df.show()
filtered_vine_helpful_comments_df = vine_helpful_comments_df.filter(vine_helpful_comments_df.fraction_helpful_votes>=0.5)
filtered_vine_helpful_comments_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+----------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|fraction_helpful_votes|
+--------------+-----------+-------------+-----------+----+-----------------+----------------------+
|R14W2VCHHK5V7W|          5|           19|         20|   N|                Y|                  0.95|
|R1S3T57O3OYT5S|          5|           19|         20|   N|                Y|                  0.95|
|R1BTWIBLYYVOV7|          5|           30|         30|   N|                Y|                   1.0|
| R6F9VY91ADPLA|          1|            8|         30|   N|                N|   0.26666666666666666|
|R3PXNV89DFIXKV|          5|           35|         37|   N|                Y|    0.9459459459459459|
|R2ZF9NYVT3J7D6|          5|           19|         22|   N|                Y|    0.8636363636363636|
|R20NYA5V0UF9NE|          5|           27|         28|   N|                Y|    0.96428571

In [78]:
#Filter the new  data vine_helpful_comments_df with the comments made by the vine program
vine_paidhelpful_comments_df = filtered_vine_helpful_comments_df.filter(filtered_vine_helpful_comments_df.vine == 'Y')
vine_paidhelpful_comments_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+---------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|percent_helpful_votes|
+--------------+-----------+-------------+-----------+----+-----------------+---------------------+
|R1B7M0OP3UNP6O|          5|           49|         52|   Y|                N|   0.9423076923076923|
|R2UUV4UGGYMQG8|          5|           34|         39|   Y|                N|   0.8717948717948718|
| R9K0LZV2BK9YY|          4|           37|         39|   Y|                N|   0.9487179487179487|
|R2OVFLNEUEGTJM|          3|           18|         25|   Y|                N|                 0.72|
| RBE09ELJ77LQ0|          5|           44|         45|   Y|                N|   0.9777777777777777|
+--------------+-----------+-------------+-----------+----+-----------------+---------------------+
only showing top 5 rows



In [68]:
#Filter the new  data vine_helpful_comments_df with the comments not made by the vine program
nonvine_comments_df = filtered_vine_helpful_comments_df.filter(filtered_vine_helpful_comments_df.vine == 'N')
nonvine_comments_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+---------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|percent_helpful_votes|
+--------------+-----------+-------------+-----------+----+-----------------+---------------------+
|R14W2VCHHK5V7W|          5|           19|         20|   N|                Y|                 0.95|
|R1S3T57O3OYT5S|          5|           19|         20|   N|                Y|                 0.95|
|R1BTWIBLYYVOV7|          5|           30|         30|   N|                Y|                  1.0|
|R3PXNV89DFIXKV|          5|           35|         37|   N|                Y|   0.9459459459459459|
|R2ZF9NYVT3J7D6|          5|           19|         22|   N|                Y|   0.8636363636363636|
|R20NYA5V0UF9NE|          5|           27|         28|   N|                Y|   0.9642857142857143|
|R2X8FZRUOS8R8C|          4|           25|         26|   N|                Y|   0.9615384615384616|


In [86]:
#determine the total number of reviews for the paid vine program
total_count_of_reviews_vine = vine_paidhelpful_comments_df.count()
print('total number of reviews for the paid vine program', total_count_of_reviews_vine)
#determine the total number of 5-star reviews for the paid vine program
total_5star_reviews_vine = vine_paidhelpful_comments_df.filter(filtered_vine_helpful_comments_df.star_rating == 5).count()
print('total number of 5-star reviews for the paid vine program', total_5star_reviews_vine)
#determine the percentage of 5 star reviews for the paid vine program
percentage_of_5star_reviews_vine  = (total_5star_reviews_vine/total_count_of_reviews_vine)*100
print('total percentage of 5-star reviews for the paid vine program %', percentage_of_5star_reviews_vine)

#determine the total number of reviews for the non vine program
total_count_of_non_vine = nonvine_comments_df.count()
print('total number of reviews for the non vine program', total_count_of_non_vine)
#determine the total number of 5-star reviews for the non vine program
total_5star_reviews_non_vine = nonvine_comments_df.filter(nonvine_comments_df.star_rating == 5).count()
print('total number of 5-star reviews for the non vine program', total_5star_reviews_non_vine)
#determine the percentage of 5 star reviews for the non vine program
percentage_of_5star_reviews  = (total_5star_reviews_non_vine/total_count_of_non_vine)*100
print('total percentage of 5-star reviews for the non vine program %', percentage_of_5star_reviews)

total number of reviews for the paid vine program 47
total number of 5-star reviews for the paid vine program 15
total percentage of 5-star reviews for the paid vine program % 31.914893617021278
total number of reviews for the non vine program 8362
total number of 5-star reviews for the non vine program 4332
total percentage of 5-star reviews for the non vine program % 51.80578808897393
